In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from dotenv import load_dotenv
import warnings

In [3]:
load_dotenv()
vectordb_path = "./"
resumes_path = "docs"
huggingface_api = os.getenv("hugging_api_key")

In [4]:
### In this code first we are loading the documents from Folder. Here you have
### to provide the folder path and it will load the all documents.
docs_loader = DirectoryLoader("/home/stackops/langchain-labs/data/vks/pdf/vi")
documents = docs_loader.load()

/home/stackops/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
### After Loading the documents we are splitting the documents into chunk with the help of RecursiveCharacterTextSplitter.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [6]:
### In this code we are using embeddings model to convert the chunk of document to vectors
embeddings = SentenceTransformerEmbeddings(model_name="paraphrase-multilingual-mpnet-base-v2")

/tmp/ipykernel_25178/3208077349.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="paraphrase-multilingual-mpnet-base-v2")


In [7]:
### This code will convert the docs to vectors and store the vectors into vector db.
### Here we are using Chroma db as a vector database.
### The Persist function will help us to store the vector db in local disk.
vectordb = Chroma.from_documents(documents= docs,
                    embedding= embeddings,
                    persist_directory= vectordb_path)
print("Documents stored in the database")


Documents stored in the database


In [8]:
import os
import weave
import asyncio
import json
import warnings
from groq import Groq
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from weave import Model
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage



In [9]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/home/stackops/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

In [10]:
weave.init(project_name="genai-application")
persist_directory = "./"
client = ChatVertexAI(model="gemini-1.5-flash")


Logged in as Weights & Biases user: cuongpigerr.
View Weave data at https://wandb.ai/cuongpigerr-vng-cloud/genai-application/weave


In [11]:
@weave.op()
def generate_solution_with_llm(query: str, vector_db_content: str, model: str ):
    user_input = f"User query: {query}. Retrieved relevant data: {vector_db_content}"

    chat_completion = client.invoke(
        [
            SystemMessage("You are a helpful assistant. Based on the query and content, provide the solution."),
            HumanMessage(user_input),
        ]
    )
    
    return chat_completion.content

In [12]:
class RAGModel(Model):
    model_name: str = "llama3-8b-8192"

    @weave.op()
    def predict(self, question: str) -> dict:
        embeddings = SentenceTransformerEmbeddings(model_name="paraphrase-multilingual-mpnet-base-v2")
        vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
        vectorstore_retriever = vectordb.as_retriever(search_kwargs={"k": 3})
        retrieved_documents = vectorstore_retriever.get_relevant_documents(question)
        retrieved_content = " ".join([doc.page_content for doc in retrieved_documents])
        answer = generate_solution_with_llm(
            query=question,
            vector_db_content=retrieved_content,
            model=self.model_name,
        )
        return {'answer': answer, 'context': retrieved_content}

In [13]:
# Define your scoring function
@weave.op()
async def context_precision_score(question, model_output):
    context_precision_prompt = """Given the question, answer, and context, verify if the context was useful in arriving at the given answer. Give a verdict as "1" if useful and "0" if not, with RAW-TEXT JSON output.
Output in only valid RAW-TEXT JSON format.

question: {question}
context: {context}
answer: {answer}
verdict: """

    prompt = context_precision_prompt.format(
        question=question,
        context=model_output['context'],
        answer=model_output['answer'],
    )

    response = client.invoke(
        [
            HumanMessage(prompt),
        ]
    )
    
    response_message = response.content.strip()
    print(response_message)

    try:
        response_json = json.loads(response_message)
        verdict = int(response_json["verdict"]) == 1
    except (json.JSONDecodeError, KeyError, ValueError):
        verdict = False

    return {"verdict": verdict}

In [14]:
questions = [
    {"question": "VKS là gì?"},
]

model = RAGModel()

evaluation = weave.Evaluation(dataset=questions, scorers=[context_precision_score])


Using 'model_output' key for compatibility with older scorers. Please update scorers to use 'output' parameter.


In [15]:
await evaluation.evaluate(model)

🍩 https://wandb.ai/cuongpigerr-vng-cloud/genai-application/r/call/01953634-ecf6-7431-98f8-3a88c9b2bf86
🍩 https://wandb.ai/cuongpigerr-vng-cloud/genai-application/r/call/01953634-ecf3-74d2-8e45-6c03e95f98c2
🍩 https://wandb.ai/cuongpigerr-vng-cloud/genai-application/r/call/01953634-ecf4-76a0-af83-e79f78a341c5


/tmp/ipykernel_25178/1554766186.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
/tmp/ipykernel_25178/1554766186.py:9: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_documents = vectorstore_retriever.get_relevant_documents(question)


🍩 https://wandb.ai/cuongpigerr-vng-cloud/genai-application/r/call/01953634-fbd6-7793-9849-26fc710337d8
🍩 https://wandb.ai/cuongpigerr-vng-cloud/genai-application/r/call/01953634-fbfb-7321-9264-d1115fffe53a
🍩 https://wandb.ai/cuongpigerr-vng-cloud/genai-application/r/call/01953634-fbfd-7753-b8be-ccaefe7b385b
🍩 https://wandb.ai/cuongpigerr-vng-cloud/genai-application/r/call/01953635-0320-7030-85b0-4d3ba44cb679
🍩 https://wandb.ai/cuongpigerr-vng-cloud/genai-application/r/call/01953635-0321-7262-8b0c-3e4a30262bdf
{"verdict": "1"}


Evaluated 1 of 1 examples

Evaluation summary
{
    'context_precision_score': {'verdict': {'true_count': 1, 'true_fraction': 1.0}},
    'model_latency': {'mean': 5.674119472503662}
}

{'context_precision_score': {'verdict': {'true_count': 1,
   'true_fraction': 1.0}},
 'model_latency': {'mean': 5.674119472503662}}

🍩 https://wandb.ai/cuongpigerr-vng-cloud/genai-application/r/call/01953635-0512-7c63-aee9-e982436358c8


![](./tmp.png)